In [1]:
import numpy as np
from sklearn import datasets

In [2]:
# 使用手写识别数据作为样本数据
digits = datasets.load_digits()
X = digits.data
y = digits.target

In [3]:
# 先使用train_test_split将样本数据拆分为训练数据集和测试数据集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [4]:
X_train.shape

(1078, 64)

In [6]:
y_test.shape

(719,)

In [8]:
# KNN算法中使用训练数据集和测试数据集进行超参数k和p的调整
from sklearn.neighbors import KNeighborsClassifier

best_score, best_k, best_p = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score, best_k, best_p = score, k, p

print("Best Score =", best_score)
print("Best k = ", best_k)
print("Best p =", best_p)

Best Score = 0.986091794159
Best k =  3
Best p = 4


## 交叉验证方法

In [9]:
# 导入Scikit Learn中交叉验证的函数
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([ 0.98895028,  0.97777778,  0.96629213])

In [10]:
# KNN算法中使用训练数据集和验证数据集进行超参数k和p的调整
from sklearn.neighbors import KNeighborsClassifier

best_score, best_k, best_p = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        # 对每个k值，p值的组合实例化KNN分类器，通过交叉验证发训练出若干个模型，然后对这若干个模型的评分求平均值，该平均值即为每次的模型评分，每次将最好的评分和对应的k，p值记录下来，最终找到评分最好的k和p值
        scores = cross_val_score(knn_clf, X_train, y_train)
        # 对若干个模型的评分求平均值
        score = np.mean(scores)
        if score > best_score:
            best_score, best_k, best_p = score, k, p

print("Best Score =", best_score)
print("Best k = ", best_k)
print("Best p =", best_p)

Best Score = 0.982359987401
Best k =  2
Best p = 2


In [11]:
knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=2, p=2)
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance')

In [12]:
knn_clf.score(X_test, y_test)

0.98052851182197498

## 回顾网格搜索

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
param_grid = [
    {
        "weights": ['distance'],
        "n_neighbors": [i for i in range(2, 11)],
        "p": [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  1.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'weights': ['distance'], 'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [15]:
grid_search.best_score_

0.98237476808905377

In [16]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [17]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.98052851182197498